In [ ]:
import os
from Bio import AlignIO
from Bio.Seq import Seq 
from Bio import SeqIO
from Bio.Align import MultipleSeqAlignment
from Bio.Align.Applications import ClustalOmegaCommandline
from Bio.Align.Applications import ClustalwCommandline
f="ERAP2allotype_withSplicedJulyNewNames1.fa"
records = []
for record in SeqIO.parse(f, format='fasta'):
    records.append(record)

In [ ]:
input_file = "ERAP2allotype_withSplicedJulyNewNames1.fasta"
output_file = "ERAP2allotype_withSplicedJulyNewNames1.aln"
#Running ClustalW2 for multiple sequence alignment
clustalw_cline = ClustalwCommandline("clustalw2", infile=input_file, outfile=output_file)
stdout, stderr = clustalw_cline()

In [ ]:
from skbio import TabularMSA, Protein
from skbio import DistanceMatrix
from skbio.sequence.distance import hamming
from skbio.tree import nj
import ete3
from ete3 import Tree
msa = TabularMSA.read('ERAP2allotype_withSplicedJulyNewNames1.fasta', constructor=Protein)
msa.reassign_index(minter='id')
dm = DistanceMatrix.from_iterable(msa, metric=hamming, keys=msa.index)
tree = nj(dm)
ts = ete3.TreeStyle()
ts.show_branch_length = True
newick_str = nj(dm, result_constructor=str)
t = Tree(newick_str)
new_root = tree.find('ERAP2*00*01').ancestors()[0]  #root (ancestral node)
rooted_tree = tree.root_at(new_root)
t.write(format=1, outfile="ERAP2Allotypesete_tree.nw")

In [ ]:
from ete3 import PhyloTree
os.environ['QT_QPA_PLATFORM']='offscreen'
%matplotlib inline
def attach_sequences_to_tree(tree, alignment):
    sequence_dict = {seq.id: seq for seq in alignment}
    for leaf in tree:
        sequence_id = leaf.name  
        if sequence_id in sequence_dict:
            leaf.add_features(sequence=sequence_dict[sequence_id])
        else:
            print(f"Sequence not found for node {leaf.name}")
if __name__ == "__main__":
    tree_file = "ERAP2Allotypesete_tree.nw" 
    alignment_file = "ERAP2allotype_withSplicedJuly.aln"  
    gene_tree = Tree(tree_file)
    alignment = AlignIO.read(alignment_file, "clustal") 
    attach_sequences_to_tree(gene_tree, alignment)
    ts = TreeStyle()
    gene_tree.render('/hpc/dla_lti/araja/hapsnew/ERAP2/ERAP2treeete3PhyloMSAtest1.png', tree_style=ts) #checking msa

In [ ]:
from ete3 import Tree, TreeStyle
t = Tree("ERAP2Allotypesete_tree.nw")
t.set_outgroup('ERAP2*00*01')
os.environ['QT_QPA_PLATFORM']='offscreen'
%matplotlib inline
ts = ete3.TreeStyle()
ts.show_branch_length = True
#circular_style = TreeStyle()
#circular_style.mode = "c" 
#circular_style.scale = 20
#t.render('ERAP2treeete3test.png', w=183, units="mm", tree_style=circular_style)
t.render('ERAP2treeete3NewNames1.svg', tree_style=ts)